# Transfer Learning Model 'Xception Dropout' #

In [1]:

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet201, preprocess_input

from keras import backend as K


Using TensorFlow backend.


In [2]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir, 'train')
     valid_path = os.path.join(base_dir, 'valid')

     # instantiate data generators
     datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

# the size of the images in the PCAM dataset
IMAGE_SIZE = 96
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
train_gen, val_gen = get_pcam_generators('C:\\Users\\s169753\\OneDrive - TU Eindhoven\\TUe\\Jaar 3\\Q3\\Project Imaging\\train+val')
input = Input(input_shape)
# get the pretrained model, cut out the top layer
pretrained = DenseNet201(input_shape=input_shape, include_top=False, weights='imagenet')

#for layer in pretrained.layers[-4]:
#    layer.trainable = False

# Freeze the layers except the last 4 layers
#for i, layer in enumerate(pretrained.layers):
    #layer.trainable = False
#    print(layer.name)

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [3]:
layernames = []
for i,layer in enumerate(pretrained.layers):
    layernames.append(layer.name)

for dropvalue in np.arange(0,1,0.05):
    dropvalue = np.around(dropvalue,2)
    pretrained = DenseNet201(input_shape=input_shape, include_top=False, weights='imagenet')

    output = pretrained(input)
    output = GlobalAveragePooling2D()(output)
    output = Dropout(dropvalue)(output)
    output = Dense(1, activation='sigmoid')(output)

    # Check the trainable status of the individual layers
    #for layer in pretrained.layers:
    #    print(layer, layer.trainable)

    model = Model(input, output)

    # note the lower lr compared to the cnn example
    model.compile(SGD(lr=0.001, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])
        
    # save the model and weights
    string = 'DropOut'+str(dropvalue)
    print(string)
    model.summary()
    
    model_name = 'Densenet_Dropout'+string
    model_filepath = model_name + '.json'
    weights_filepath = model_name + '_weights.hdf5'

    model_json = model.to_json() # serialize model to JSON
    with open(model_filepath, 'w') as json_file:
        json_file.write(model_json)


    # define the model checkpoint and Tensorboard callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(os.path.join('logs', model_name))
    callbacks_list = [checkpoint, tensorboard]


    # train the model, note that we define "mini-epochs"
    train_steps = train_gen.n//train_gen.batch_size//20
    val_steps = val_gen.n//val_gen.batch_size//20

    # since the model is trained for only 10 "mini-epochs", i.e. half of the data is
    # not used during training

    history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=10,
                        callbacks=callbacks_list)
    K.clear_session()
    

DropOut0.0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
densenet201 (Model)          (None, 3, 3, 1920)        18321984  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1920)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1921      
Total params: 18,323,905
Trainable params: 18,094,849
Non-trainable params: 229,056
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 3912s 17s/step - loss: 0.3854 - acc: 0.8326 - val_loss: 0.4

KeyboardInterrupt: 